In [1]:
import sys 
import numpy as np # linear algebra
from scipy.stats import randint
import matplotlib.pyplot as plt # this is used for the plot the graph 
%matplotlib inline
import re
from tqdm import notebook
import seaborn as sns
import tensorflow as tf
from scipy import stats
import pandas as pd
import json
from pandas.io.json import json_normalize

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [14]:
column_names = ['MPG','Cylinders','Displacement','Horsepower','Weight',
                'Acceleration', 'Model Year', 'Origin'] 
raw_dataset = pd.read_csv('auto-mpg.data', names=column_names,
                      na_values = "?", comment='\t',
                      sep=" ", skipinitialspace=True)

dataset = raw_dataset.copy()
dataset.tail()

,MPG,Cylinders,Displacement,Horsepower,Weight,Acceleration,Model Year,Origin
393,27.0,4,140.0,86.0,2790.0,15.6,82,1
394,44.0,4,97.0,52.0,2130.0,24.6,82,2
395,32.0,4,135.0,84.0,2295.0,11.6,82,1
396,28.0,4,120.0,79.0,2625.0,18.6,82,1
397,31.0,4,119.0,82.0,2720.0,19.4,82,1


In [15]:
##drop missingness
dataset.isna().sum()
dataset=dataset.dropna()

In [16]:
## categorical
ori=dataset.pop('Origin')
stats.describe(ori)

DescribeResult(nobs=392, minmax=(1, 3), mean=1.5765306122448979, variance=0.6488595438175271, skewness=0.911679089938487, kurtosis=-0.8464526068608298)

In [17]:
ori1=1.*(ori==1)
ori2=1.*(ori==2)

In [18]:
dataset['origin1']=ori1
dataset['origin2']=ori2

### split into training and testing

In [19]:
train=dataset.sample(frac=0.6,random_state=123)
test=dataset.drop(train.index)

trainy=train.pop('MPG')
testy=test.pop('MPG')

train=np.array(train)
test=np.array(test)
trainy=np.array(trainy).reshape(-1,1)
testy=np.array(testy).reshape(-1,1)


### Normalize the space

In [27]:
##define a normalization function
def norm_data(train, test,norm=True):
    std_train=np.ones(train.shape[1])
    mu_train=np.zeros(train.shape[1])
    if(norm):
        std_train = np.std(train, 0)
        mu_train=np.mean(train,0)

    train=(train - np.full(train.shape, mu_train)) / \
            np.full(train.shape, std_train)
    test=(test - np.full(test.shape, mu_train)) / \
            np.full(test.shape, std_train)
    return train,test,mu_train,std_train


train_x,test_x,_,_=norm_data(train[:,:-2],test[:,:-2],norm=True)
train_y,test_y,_,_=norm_data(trainy,testy,norm=True)

In [29]:
train_x=np.c_[train_x,train[:,-2:]]
test_x=np.c_[test_x,test[:,-2:]]

### Output data

In [35]:
Train=np.c_[train_x,train_y]
Test=np.c_[test_x,test_y]


In [36]:
np.save('Train',Train)
np.save('Test',Test)
